### Duplicate checker
----
Checks `C:\Users\kade\Pictures` for duplicate furry porn.

In [ ]:
import os
import hashlib
from PyQt5.QtWidgets import QApplication, QMainWindow, QPushButton, QGroupBox, QHBoxLayout, QVBoxLayout, QTextEdit, QWidget, QScrollArea

class DuplicateFinder(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Duplicate Finder")
        self.setGeometry(100, 100, 800, 600)

        self.central_widget = QWidget(self)
        self.setCentralWidget(self.central_widget)

        self.layout = QVBoxLayout(self.central_widget)

        self.scroll_area = QScrollArea(self)
        self.scroll_area.setWidgetResizable(True)

        self.scroll_content = QWidget(self.scroll_area)
        self.group_box_layout = QVBoxLayout(self.scroll_content)
        self.scroll_area.setWidget(self.scroll_content)

        self.layout.addWidget(self.scroll_area)

        self.delete_button = QPushButton("Delete All Duplicates", self)
        self.delete_button.clicked.connect(self.delete_all_duplicates)
        self.layout.addWidget(self.delete_button)

        self.find_button = QPushButton("Find Duplicates", self)
        self.find_button.clicked.connect(self.find_duplicates)
        self.layout.addWidget(self.find_button)

        # Dictionary to store file hashes and paths in memory
        self.file_hashes = {}

        # Start by scanning the default directory
        self.find_duplicates()

    def calculate_md5(self, file_path):
        hash_md5 = hashlib.md5()
        with open(file_path, "rb") as f:
            for chunk in iter(lambda: f.read(4096), b""):
                hash_md5.update(chunk)
        return hash_md5.hexdigest()

    def find_duplicates(self):
        directory = "C:\\Users\\kade\\Pictures"
        if directory:
            # Clear existing data when finding duplicates
            self.clear_group_boxes()
            self.file_hashes = {}

            for root, dirs, files in os.walk(directory):
                for file in files:
                    file_path = os.path.join(root, file).replace('/', '\\')
                    file_hash = self.calculate_md5(file_path)

                    if file_hash in self.file_hashes:
                        # Duplicate found
                        self.create_duplicate_group_box(self.file_hashes[file_hash], file_path)
                    else:
                        self.file_hashes[file_hash] = file_path

    def clear_group_boxes(self):
        while self.group_box_layout.count():
            item = self.group_box_layout.takeAt(0)
            if item.widget():
                item.widget().deleteLater()

    def create_duplicate_group_box(self, path1, path2):
        group_box = QGroupBox(self)
        layout = QHBoxLayout(group_box)

        text_edit1 = QTextEdit(path1, self)
        text_edit1.setReadOnly(True)
        layout.addWidget(text_edit1)

        text_edit2 = QTextEdit(path2, self)
        text_edit2.setReadOnly(True)
        layout.addWidget(text_edit2)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(lambda: self.delete_duplicate(text_edit1, delete_button, path1, path2))
        layout.addWidget(delete_button)

        self.group_box_layout.addWidget(group_box)

    def delete_duplicate(self, text_edit, delete_button, path1, path2):
        group_box = text_edit.parent()
        group_box.layout().removeWidget(text_edit)
        group_box.layout().removeWidget(delete_button)
        text_edit.deleteLater()
        delete_button.deleteLater()

        # Determine which path was deleted
        deleted_path = path1 if text_edit.toPlainText() == path1 else path2
        print(f"Deleted: {deleted_path}")

        try:
            # Actually delete the file from the filesystem
            os.remove(deleted_path)
        except FileNotFoundError:
            print(f"File not found: {deleted_path}")

    def delete_all_duplicates(self):
        deleted_paths = set()  # Keep track of deleted paths

        for i in range(self.group_box_layout.count()):
            group_box = self.group_box_layout.itemAt(i).widget()
            for child in group_box.children():
                if isinstance(child, QTextEdit):
                    path = child.toPlainText()
                    if path not in deleted_paths:
                        try:
                            # Actually delete the file from the filesystem
                            os.remove(path)
                            print(f"Deleted: {path}")
                            deleted_paths.add(path)  # Add the deleted path to the set
                        except FileNotFoundError:
                            print(f"File not found: {path}")

        # Clear the UI after deletion
        self.clear_group_boxes()

if __name__ == "__main__":
    app = QApplication([])
    window = DuplicateFinder()
    window.show()
    app.exec_()

### Rename files to their MD5 checksum.
----
`C:\Users\kade\Desktop\MD5 RENAME`

In [4]:
import os
import hashlib

def calculate_md5(file_path):
    hash_md5 = hashlib.md5()
    with open(file_path, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

def rename_files(folder_path):
    md5_mapping = {}  # Store MD5 hashes to check for duplicates
    duplicates = []   # Store duplicate files

    for root, dirs, files in os.walk(folder_path):
        for file in files:
            file_path = os.path.join(root, file)
            md5_hash = calculate_md5(file_path)
            _, file_extension = os.path.splitext(file)
            new_file_name = md5_hash + file_extension
            new_file_path = os.path.join(root, new_file_name)

            # Check if the new file already exists
            if md5_hash in md5_mapping:
                # Handle duplicate by renaming with a suffix
                duplicate_count = md5_mapping[md5_hash]
                md5_mapping[md5_hash] += 1
                new_file_name = f"{md5_hash}_{duplicate_count}{file_extension}"
                new_file_path = os.path.join(root, new_file_name)

                # Add duplicate to the list
                duplicates.append((file, new_file_name))

            # Store the MD5 hash in the mapping
            md5_mapping.setdefault(md5_hash, 1)

            os.rename(file_path, new_file_path)
            print(f'Renamed: {file} to {new_file_name}')

    # Print list of duplicates
    if duplicates:
        print("\nList of Duplicates:")
        for original, duplicate in duplicates:
            print(f'{original} -> {duplicate}')

folder_path = r'C:\Users\kade\Desktop\MD5 RENAME'
rename_files(folder_path)

Renamed: 00000007-01200130.png to cd8e98a945e6d46932eb9712940c31b1.png
Renamed: 00000008-01200137.png to ca5203b8fc0b4b13945ab7d98e42e332.png
Renamed: 00000009-01200144.png to 57ec27b140c95e757783712160f1e344.png
Renamed: 00000010-01200148.png to 9cff27b8fffa0c5672cdb270c756e2c0.png
Renamed: 00000011-01200156.png to 77ec13fac0f7c80a75db80892251a3bc.png
Renamed: 185b54500b0039a1ffdb8fd21e75d2ac.gif to 524a321e749e3bd36ac271ccee8e8f2c.gif
Renamed: 5d5876d3-7b18-4ec8-b51e-074faf6c93a5.png to 408a014c3976043397189916294c79e9.png
Renamed: aeca5800-4e3b-4a72-95e5-5856efdfe645.png to bf592148431da68d3899ce03e99a56e4.png
Renamed: cce916af-0f43-49aa-8b40-1afae0c21d1a.png to c1d68d0a1c81b0ec6af14b1c6da63d69.png
Renamed: chunie.png to 003f4307e6434d1123dbf57925bdc615.png
Renamed: darkgem.png to 1b5537e9f0d47524a2a3ca1a4314d0c1.png
Renamed: flying_with_you_by_himmis_dfs6890-pre.jpg to 7dfde46651cf354a0c583d6e195de74c.jpg
Renamed: honovy.png to 8922e6956b71aaa7023a1604b735479c.png
Renamed: image_0_

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\kade\\Desktop\\MD5 RENAME\\wtf\\0000041.png' -> 'C:\\Users\\kade\\Desktop\\MD5 RENAME\\wtf\\1d8c37de70a33cbfbcfd5d8804a03a0c.png'

### Alphabetize text
----
`C:\Users\kade\Desktop\ALPHABETIZE THIS.txt`

In [4]:
import pyperclip

file_path = r'C:\Users\kade\Desktop\ALPHABETIZE THIS.txt'

# Read lines from the file
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Alphabetize the lines
alphabetized_lines = sorted(lines)

# Join the lines into a single string
result = ''.join(alphabetized_lines)

# Copy the result to the clipboard
pyperclip.copy(result)

### Calculate SHA1 checksum
----

In [1]:
import hashlib
import sys
from tkinter import Tk
from tkinter.filedialog import askopenfilename

def calculate_sha1(file_name):
    sha1 = hashlib.sha1()
    with open(file_name, "rb") as file:
        while True:
            data = file.read(65536)
            if not data:
                break
            sha1.update(data)
    return sha1.hexdigest()

def select_file():
    root = Tk()
    root.withdraw()  # Hide the main window

    file_name = askopenfilename(title="Select a file")

    if not file_name:
        print("No file selected. Exiting.")
        sys.exit(1)

    return file_name

file_name = select_file()
sha1sum = calculate_sha1(file_name)

print(sha1sum)


cd19722d99ac133450580445eaf2084f79b8d59b


### Calculate MD5 checksum
---

In [ ]:
import hashlib
import sys
from tkinter import Tk
from tkinter.filedialog import askopenfilename

def calculate_md5(file_name):
    md5 = hashlib.md5()
    with open(file_name, "rb") as file:
        while True:
            data = file.read(65536)
            if not data:
                break
            md5.update(data)
    return md5.hexdigest()

def select_file():
    root = Tk()
    root.withdraw()  # Hide the main window

    file_name = askopenfilename(title="Select a file")

    if not file_name:
        print("No file selected. Exiting.")
        sys.exit(1)

    return file_name

file_name = select_file()
md5sum = calculate_md5(file_name)

print(md5sum)

### Calculate SHA256 checksum
----

In [2]:
import hashlib
import sys
from tkinter import Tk
from tkinter.filedialog import askopenfilename

def calculate_sha256(file_name):
    sha256 = hashlib.sha256()
    with open(file_name, "rb") as file:
        while True:
            data = file.read(65536)
            if not data:
                break
            sha256.update(data)
    return sha256.hexdigest()

def select_file():
    root = Tk()
    root.withdraw()  # Hide the main window

    file_name = askopenfilename(title="Select a file")

    if not file_name:
        print("No file selected. Exiting.")
        sys.exit(1)

    return file_name

file_name = select_file()
sha256sum = calculate_sha256(file_name)

print(sha256sum)

136be29906671d18d7a0d696c1aa01a3e9e45d70b79e52716406be043ec93ccb
